In [3]:
# import libraries
import multiprocessing as mp
import geopandas as gpd
import osmnx as ox
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from shapely import wkt
from warnings import warn
from heapq import heappop, heappush
from itertools import count

In [4]:
# generate G of LA County
G = ox.graph_from_place('Los Angeles County, CA, USA', network_type='drive')

/Users/kweon-youngseo/miniforge3/envs/ppd599/lib/python3.10/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/Users/kweon-youngseo/miniforge3/envs/ppd599/lib/python3.10/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/Users/kweon-youngseo/miniforge3/envs/ppd599/lib/python3.10/site-packages/shapely/set_operations.py:340: RuntimeWarning: invalid value encountered in union
  return lib.union(a, b, **kwargs)
/Users/kweon-youngseo/miniforge3/envs/ppd599/lib/python3.10/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/Users/kweon-youngseo/miniforge3/envs/ppd599/lib/python3.10/site-packages/shapely/set_operations.py:340: RuntimeWarning: invalid value encountered in union
  return lib.union(a, b, **kwargs)
/Users/kweon-youngseo/miniforge3/envs/ppd599/lib

In [5]:
# add attributes
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)
G = ox.bearing.add_edge_bearings(G)

In [6]:
# create gdfs
nodes_gdf, edges_gdf = ox.graph_to_gdfs(G)

In [7]:
# import O,D
origins_df = pd.read_csv("../Data/origins_gdf.csv")
destinations_df = pd.read_csv("../Data/destinations_gdf.csv")

origins_df['geometry'] = origins_df['geometry'].apply(wkt.loads)
destinations_df['geometry'] = destinations_df['geometry'].apply(wkt.loads)

# geodataframe of O, D
origins_gdf = gpd.GeoDataFrame(origins_df, geometry='geometry')
destinations_gdf = gpd.GeoDataFrame(destinations_df, geometry='geometry')

# Add attributes to G edges

Use add_edge_traffic_times function to add extra time based on traffic controls

In [8]:
from functions import add_edge_traffic_times

In [9]:
G = add_edge_traffic_times(G, traffic_signals_time = 30, stop_time = 15, turning_circle_time = 5, crossing_time = 5, give_way_time = 5, mini_roundabout_time = 5)

In [10]:
# create gdfs
nodes_gdf, edges_gdf = ox.graph_to_gdfs(G)

In [11]:
edges_gdf

osmid   
u           v           key                                                    
653656      1718677597  0                                          159763316  \
            123189012   0    [1094670073, 1094670074, 1055876622, 383059215]   
            122697159   0     [1050285331, 1050285330, 399239123, 399239118]   
            1718756337  0                             [399258376, 399258374]   
653681      26427612    0                              [49287146, 149003055]   
...                                                                      ...   
11174591011 2052207882  0                                          194714649   
            2052207904  0                                          194714650   
            2452098356  0                                          194714650   
11174591012 11174591011 0                                          194714649   
            2052207835  0                                          237284178   

                                 lanes                       name   
u           v           key                                         
653656      1718677597  0            4       West Florence Avenue  \
            123189012   0    [5, 4, 6]  West Manchester Boulevard   
            122697159   0    [5, 4, 6]     West Manchester Avenue   
            1718756337  0       [5, 4]         Aviation Boulevard   
653681      26427612    0       [5, 6]          San Diego Freeway   
...                                ...                        ...   
11174591011 2052207882  0          NaN               Baywood Lane   
            2052207904  0          NaN              Victoria Lane   
            2452098356  0          NaN              Victoria Lane   
11174591012 11174591011 0          NaN               Baywood Lane   
            2052207835  0          NaN               Baywood Lane   

                                 highway maxspeed  oneway reversed   length   
u           v           key                                                   
653656      1718677597  0      secondary   40 mph   False    False  253.852  \
            123189012   0        primary      NaN   False    False  159.909   
            122697159   0        primary      NaN   False     True  160.976   
            1718756337  0       tertiary      NaN   False    False  402.735   
653681      26427612    0       motorway   65 mph    True    False  252.266   
...                                  ...      ...     ...      ...      ...   
11174591011 2052207882  0    residential      NaN   False     True   88.818   
            2052207904  0    residential      NaN   False    False  143.875   
            2452098356  0    residential      NaN   False     True   40.741   
11174591012 11174591011 0    residential      NaN   False     True    3.463   
            2052207835  0    residential      NaN    True    False   69.551   

                                                                      geometry   
u           v           key                                                      
653656      1718677597  0    LINESTRING (-118.37705 33.95990, -118.37705 33...  \
            123189012   0    LINESTRING (-118.37705 33.95990, -118.37695 33...   
            122697159   0    LINESTRING (-118.37705 33.95990, -118.37720 33...   
            1718756337  0    LINESTRING (-118.37705 33.95990, -118.37705 33...   
653681      26427612    0    LINESTRING (-118.40031 33.98940, -118.40087 33...   
...                                                                        ...   
11174591011 2052207882  0    LINESTRING (-118.55984 34.43007, -118.55992 34...   
            2052207904  0    LINESTRING (-118.55984 34.43007, -118.55977 34...   
            2452098356  0    LINESTRING (-118.55984 34.43007, -118.56000 34...   
11174591012 11174591011 0    LINESTRING (-118.55982 34.43004, -118.55984 34...   
            2052207835  0    LINESTRING (-118.55982 34.43004, -118.55978 34...   

                             speed_kp

In [12]:
edges_gdf[['travel_time', 'traffic_time', 'total_time']]

travel_time  traffic_time  total_time
u           v           key                                       
653656      1718677597  0           14.2             0        14.2
            123189012   0            9.5             0         9.5
            122697159   0            9.6             0         9.6
            1718756337  0           29.9            30        59.9
653681      26427612    0            8.7             0         8.7
...                                  ...           ...         ...
11174591011 2052207882  0            7.9             0         7.9
            2052207904  0           12.8             0        12.8
            2452098356  0            3.6             0         3.6
11174591012 11174591011 0            0.3             0         0.3
            2052207835  0            6.2            30        36.2

[461833 rows x 3 columns]

# Penalty functions

Use get_turn_penalty_dict function to get turn penalty dictionary

In [13]:
from functions import get_turn_penalty_dict

In [14]:
penalty = get_turn_penalty_dict(G, left_turn_penalty = 30, right_turn_penalty = 10, u_turn_penalty = 90)

In [15]:
penalty

{(122697159, 653656, 1718677597): 30,
 (122697159, 653656, 123189012): 0,
 (122697159, 653656, 122697159): 90,
 (122697159, 653656, 1718756337): 10,
 (123189012, 653656, 1718677597): 10,
 (123189012, 653656, 123189012): 90,
 (123189012, 653656, 122697159): 0,
 (123189012, 653656, 1718756337): 30,
 (1718677597, 653656, 1718677597): 90,
 (1718677597, 653656, 123189012): 30,
 (1718677597, 653656, 122697159): 10,
 (1718677597, 653656, 1718756337): 30,
 (1718756337, 653656, 1718677597): 10,
 (1718756337, 653656, 123189012): 10,
 (1718756337, 653656, 122697159): 30,
 (1718756337, 653656, 1718756337): 90,
 (61744867, 653681, 26427612): 0,
 (1717202157, 653681, 26427612): 0,
 (384413646, 668182, 361229949): 0,
 (384413646, 668182, 668183): 0,
 (668182, 668183, 359154206): 0,
 (668182, 668183, 359157443): 0,
 (452589520, 668233, 148801155): 10,
 (1717061559, 668233, 148801155): 10,
 (1719348774, 668243, 8584982268): 0,
 (1719348774, 668243, 123303478): 10,
 (2897997136, 668243, 8584982268): 30,

# shortest_path_turn_penalty

https://github.com/maxtmng/shortest_path_turn_penalty/

In [16]:
"""
    Uses Dijkstra's algorithm to find the shortest weighted paths to one or multiple targets with turn penalty.
    This function is adapted from networkx.algorithms.shortest_paths.weighted._dijkstra_multisource.
    The turn penalty implementation is based on:
    Ziliaskopoulos, A.K., Mahmassani, H.S., 1996. A note on least time path computation considering delays and prohibitions for intersection movements. Transportation Research Part B: Methodological 30, 359–367. https://doi.org/10.1016/0191-2615(96)00001-X
    Parameters
    ----------
    G : NetworkX graph
    source : non-empty iterable of nodes
        Starting nodes for paths. If this is just an iterable containing
        a single node, then all paths computed by this function will
        start from that node. If there are two or more nodes in this
        iterable, the computed paths may begin from any one of the start
        nodes.
    target : node label, single node or a list
        Ending node (or a list of ending nodes) for path. Search is halted when any target is found.
    weight: function
        Function with (u, v, data) input that returns that edge's weight
        or None to indicate a hidden edge
    penalty : dict, optional (default={})
        Dictionary containing turn penalties. The key is a tuple (u, v, m) where
        u, v are the nodes of the current edge and m is the next node.
    next_node : node, optional (default=None)
        Next node to consider from the source.
    Returns
    -------
    list of nodes
        Path from source to target.
    Raises
    ------
    NodeNotFound
        If the source or target is not in `G`.
    ValueError
        If contradictory paths are found due to negative weights.
    """

"\n    Uses Dijkstra's algorithm to find the shortest weighted paths to one or multiple targets with turn penalty.\n    This function is adapted from networkx.algorithms.shortest_paths.weighted._dijkstra_multisource.\n    The turn penalty implementation is based on:\n    Ziliaskopoulos, A.K., Mahmassani, H.S., 1996. A note on least time path computation considering delays and prohibitions for intersection movements. Transportation Research Part B: Methodological 30, 359–367. https://doi.org/10.1016/0191-2615(96)00001-X\n    Parameters\n    ----------\n    G : NetworkX graph\n    source : non-empty iterable of nodes\n        Starting nodes for paths. If this is just an iterable containing\n        a single node, then all paths computed by this function will\n        start from that node. If there are two or more nodes in this\n        iterable, the computed paths may begin from any one of the start\n        nodes.\n    target : node label, single node or a list\n        Ending node (or 

In [17]:
## used the function without any modification

def shortest_path_turn_penalty(G, source, target, weight="weight", penalty={}, next_node = None):

    G_succ = G._adj  # For speed-up (and works for both directed and undirected graphs)
    weight = nx.algorithms.shortest_paths.weighted._weight_function(G, weight)
    push = heappush
    pop = heappop
    dist = {}  # dictionary of final distances
    paths = {source: [source]}
    target_list = [target] if not isinstance(target, list) else target
    reached_target = None
    seen = {}
    c = count()
    fringe = []
    seen[source] = {}
    if next_node is None:
        for m,_ in G_succ[source].items():
            seen[source][m] = 0
            push(fringe, (0, next(c), source, m))
    else:
        push(fringe, (0, next(c), source, next_node))
    while fringe:
        (d, _, v, m) = pop(fringe)
        u = m
        if v in dist:
            if u in dist[v]:
                continue  # already searched this node.
        else:
            dist[v] = {}
        dist[v][u] = d
        if v in target_list:
            reached_target = v
            break
        e = G[v][u]
        for m in G_succ[u]:
            cost = weight(v, u, e)
            if (v,u,m) in penalty:
                cost += penalty[v,u,m]

            if cost is None:
                continue
            vu_dist = dist[v][u] + cost
            if u in dist:
                if m in dist[u]:
                    u_dist = dist[u][m]
                    if vu_dist < u_dist:
                        raise ValueError("Contradictory paths found:", "negative weights?")
            elif u not in seen or m not in seen[u] or vu_dist < seen[u][m]:
                if u not in seen:
                    seen[u] = {}
                seen[u][m] = vu_dist
                push(fringe, (vu_dist, next(c), u, m))
                if paths is not None:
                    paths[u] = paths[v] + [u]
    # The optional predecessor and path dictionaries can be accessed
    # by the caller via the pred and paths objects passed as arguments.
    return paths[reached_target]

Used get_routes_from_gdfs function to find routes of O-D pairs.

In [18]:
def get_routes_from_gdfs(G, origins_gdf, destinations_gdf, weight='weight', penalty={}):
  routes = []
  for i in range(len(origins_gdf)):
    # find nearest nodes
    orig_node = ox.distance.nearest_nodes(G, origins_gdf.iloc[i]['geometry'].x, origins_gdf.iloc[i]['geometry'].y)
    dest_node = ox.distance.nearest_nodes(G, destinations_gdf.iloc[i]['geometry'].x, destinations_gdf.iloc[i]['geometry'].y)
        
    # find routes while considering penalties
    route = shortest_path_turn_penalty(G, orig_node, dest_node, weight='weight', penalty={})
    routes.append(route)

  return routes

In [19]:
routes = get_routes_from_gdfs(G, origins_gdf, destinations_gdf, weight='total_time', penalty=penalty)

In [20]:
# travel time of 10 O-D pair routes
time_result = []

for i in range(len(routes)):
    x = sum(ox.utils_graph.get_route_edge_attributes(G, routes[i], 'total_time'))
    time_result.append(x)

In [21]:
time_result

[1465.6000000000004,
 2660.000000000001,
 1649.1,
 5657.199999999999,
 3189.000000000001,
 1644.4999999999998,
 2915.5000000000005,
 7938.700000000001,
 3121.7999999999993,
 1373.6000000000001]

In [22]:
# format time_result
def format_travel_time(seconds):
    if seconds is None:
        return None

    rounded_seconds = round(seconds)
    minutes, remaining_seconds = divmod(rounded_seconds, 60)
    hours, minutes = divmod(minutes, 60)

    formatted_time = []

    if hours:
        formatted_time.append(f"{hours} hour")

    if minutes:
        formatted_time.append(f"{minutes} min")

    if remaining_seconds:
        formatted_time.append(f"{remaining_seconds} sec")

    return ' '.join(formatted_time)

formatted_times = [format_travel_time(t) for t in time_result]

In [23]:
formatted_times

['24 min 26 sec',
 '44 min 20 sec',
 '27 min 29 sec',
 '1 hour 34 min 17 sec',
 '53 min 9 sec',
 '27 min 24 sec',
 '48 min 36 sec',
 '2 hour 12 min 19 sec',
 '52 min 2 sec',
 '22 min 54 sec']

### TEST

In [24]:
# sample
## for test, used 7th O-D pair
orig_node = ox.distance.nearest_nodes(G, origins_gdf.iloc[6]['geometry'].x, origins_gdf.iloc[6]['geometry'].y)
dest_node = ox.distance.nearest_nodes(G, destinations_gdf.iloc[6]['geometry'].x, destinations_gdf.iloc[6]['geometry'].y)

In [25]:
# with both traffic, turn penalties
shortest_path = shortest_path_turn_penalty(G, orig_node, dest_node, weight="total_time", penalty=penalty)

In [26]:
# without traffic or turn penalties
# weight = 'travel_time'
no_penalty = shortest_path_turn_penalty(G, orig_node, dest_node, weight='travel_time')

In [27]:
# only traffic penalties
only_traffic = shortest_path_turn_penalty(G, orig_node, dest_node, weight='total_time')

In [28]:
# only turn penalties
only_turn = shortest_path_turn_penalty(G, orig_node, dest_node, weight='travel_time', penalty=penalty)

In [29]:
fig, ax = ox.plot_graph(G, bgcolor='k', node_size=0, show=False)

# plot the routes
ox.plot_graph_route(G, shortest_path, route_linewidth=2, route_color='r', ax=ax, show=False)
ox.plot_graph_route(G, no_penalty, route_linewidth=2, route_color='g', ax=ax, show=False)
ox.plot_graph_route(G, only_traffic, route_linewidth=2, route_color='b', ax=ax, show=False)
ox.plot_graph_route(G, only_turn, route_linewidth=2, route_color='y', ax=ax, show=False)

plt.show()

# Calculate time

In [30]:
# calculate the total travel time of each route
no_penalties = sum(ox.utils_graph.get_route_edge_attributes(G, no_penalty, 'travel_time'))
all_penalties = sum(ox.utils_graph.get_route_edge_attributes(G, shortest_path, 'total_time'))
traffic_penalties = sum(ox.utils_graph.get_route_edge_attributes(G, only_traffic, 'total_time'))
turn_penalites = sum(ox.utils_graph.get_route_edge_attributes(G, only_turn, 'travel_time'))

In [31]:
osmnx = ox.distance.shortest_path(G, orig_node, dest_node, weight='travel_time')
osmnx_result = sum(ox.utils_graph.get_route_edge_attributes(G,osmnx, 'travel_time'))

In [32]:
data = {
    'osmnx_result': [osmnx_result],
    'no_penalties': [no_penalties],
    'all_penalties': [all_penalties],
    'traffic_penalties': [traffic_penalties],
    'turn_penalites': [turn_penalites]
}

comparison = pd.DataFrame(data)

In [33]:
comparison

,osmnx_result,no_penalties,all_penalties,traffic_penalties,turn_penalites
0,1751.3,1751.3,2629.1,2356.6,1778.9
